# ================================================
# Laboratorio 2 — Similitud de Coseno
# ================================================

# Instrucciones
1. **Carga de embeddings**  
   - Utilizar las representaciones vectoriales preentrenadas (*GloVe 100d*).  
   - Verificar que los términos requeridos estén contenidos en el vocabulario.  

2. **Cálculo de vectores de expresión**  
   - Construir expresiones vectoriales simples mediante operaciones aritméticas (suma, resta).  

3. **Medición de similitud**  
   - Calcular la similitud de coseno entre los vectores de la expresión y los vectores de referencia.  
   - Reportar:  
     - Valor de la similitud de coseno.  
     - Normas de cada vector (\(\|\mathbf{u}\|\)).  

4. **Casos de estudio**  
   Realizar caso por caso

5. **Documentación de resultados**  
   - Registrar en el notebook los valores obtenidos de similitud y normas vectoriales para cada caso.  
   - Incluir comentarios breves interpretando los resultados.  

6. **Conclusiones**  
   - Redactar conclusiones **caso por caso** sobre la utilidad de la similitud de coseno.

In [2]:
import numpy as np
from typing import List
import gensim.downloader as api

In [3]:
# ========= 1) Cargar GloVe 100D =========
model = api.load("glove-wiki-gigaword-100")

In [7]:
# ========= 2) Funciones utilitarias =========
def embedding(word: str) -> np.ndarray:
    """Vector de la palabra (lanza KeyError si no está en vocab)."""
    return model[word]

def build_vector(plus: List[str], minus: List[str]) -> np.ndarray:
    """Suma y resta embeddings explícitamente."""
    dim = model[next(iter(model.key_to_index))].shape[0]
    v = np.zeros(dim, dtype=np.float32)
    for w in plus:
        if w in model:
            v += model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    for w in minus:
        if w in model:
            v -= model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    return v

def cos(a: np.ndarray, b: np.ndarray) -> float:
    na, nb = np.linalg.norm(a), np.linalg.norm(b)
    if na == 0 or nb == 0:
        return float("nan")
    return float(np.dot(a, b) / (na * nb))

def vec_norm(v: np.ndarray) -> float:
    return float(np.linalg.norm(v))

def report_case(label, plus, minus, target_plus):
    expr_vec   = build_vector(plus, minus)
    target_vec = build_vector(target_plus, [])
    print("====", label, "====")
    print("Expresión =", " + ".join(plus) + " - " + " - ".join(minus) if minus else " + ".join(plus))
    print("Target    =", " + ".join(target_plus))
    print("cos(expr, target) =", round(cos(expr_vec, target_vec), 6))
    print("||expr||           =", round(vec_norm(expr_vec), 6))
    print("||target||         =", round(vec_norm(target_vec), 6))
    print()


def nearest_neighbors(plus, minus, topn=3):
    """
    Devuelve [(palabra, score)] usando el modelo GloVe de gensim.
    'score' es la similitud coseno calculada por el modelo.
    """
    try:
        sims = model.most_similar(positive=plus, negative=minus, topn=topn)
        return [(w, float(s)) for (w, s) in sims]
    except KeyError as e:
        print(f"[OOV] {e}")
        return []
    except Exception as e:
        print(f"[Error] {e}")
        return []

- - -

## Caso 1 — Capitales

Construya un vector que represente la relación:  
**paris + italy - france ≈ rome**

1. Defina el vector de la expresión (`expr_vec`).  
2. Defina el vector objetivo (`target_vec`).  
3. Calcule `cos(expr, target)`, `||expr||` y `||target||`.  
4. Escriba su conclusión.  

In [ ]:
# 1. Vector de la expresión: paris + italy - france
expr_vec = build_vector(['paris', 'italy'], ['france'])

# 2. Vector objetivo: rome
target_vec = embedding('rome')

# 3. Calcular similitud coseno y normas
sim = cos(expr_vec, target_vec)
norm_expr = vec_norm(expr_vec)
norm_target = vec_norm(target_vec)

print("==== Caso 1 — Capitales ====")
print("cos(expr, target) =", round(sim, 6))
print("||expr|| =", round(norm_expr, 6))
print("||target|| =", round(norm_target, 6))


==== Caso 1 — Capitales ====
cos(expr, target) = 0.8084
||expr|| = 6.227802
||target|| = 5.523363


### Conclusión

El modelo GloVe captura la relación semántica entre los países y sus capitales, el que la similitud sea mayor a 0.8 confirma que Paris es a Francias lo que roma es a Italia. Está reflejando en el espacio vectorial. Eso evidencia que la similitud de coseno es ua métrica efectiva para medir asociaciones semánticas en representaciones distribuidas de palabras.

- - -

## Caso 2 — Dictador/País

Construya un vector que represente la relación:  
**hitler + italy - germany**

1. Construya el vector de la expresión.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su interpretación de los resultados.  

In [ ]:
# 1. Construcción del vector de la expresión
expr_vec = build_vector(['hitler', 'italy'], ['germany'])

# 2. Vecinos más cercanos de la expresión
vecinos = nearest_neighbors(['hitler', 'italy'], ['germany'], topn=3)

# 3) Reportar similitud coseno con cada vecino
print("==== Caso 2 — Dictador/País ====")
print("Vecinos más cercanos y similitud con la expresión:")
res = []
for w, score_model in vecinos:
    try:
        sim_cos = cos(expr_vec, embedding(w))
    except KeyError:
        sim_cos = float('nan')
    res.append((w, score_model, sim_cos))

# Mostrar resultados ordenados tal como vienen del modelo
for w, s_model, s_cos in res:
    print(f"  - {w:12s} | most_similar={s_model:.6f} | cos(expr, {w})={s_cos:.6f}")


==== Caso 2 — Dictador/País ====
Vecinos más cercanos y similitud con la expresión:
  - mussolini    | most_similar=0.827200 | cos(expr, mussolini)=0.816139
  - fascist      | most_similar=0.685154 | cos(expr, fascist)=0.668780
  - stalin       | most_similar=0.667259 | cos(expr, stalin)=0.639061


### Conclusión

La expresión hitler + italy – germany proyecta correctamente el contexto al “trasladar” a Hitler desde Alemania hacia Italia, el modelo recupera a Mussolini y el concepto de fascismo, confirmando la analogía.

- - - 

## Caso 3 — Religión/Líder

Construya un vector que represente la relación:  
**christianity ≈ jesus**

1. Construya el vector de la palabra `christianity`.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su interpretación: ¿aparece `jesus` entre los vecinos?  

## Caso 4 — Opuestos

Analice la relación:  
**good - bad**

1. Construya el vector de la expresión `good - bad`.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su conclusión sobre si los vecinos reflejan un contraste u oposición semántica.

## Caso 5 — Propio

Defina usted mismo un caso interesante. Ejemplos:  
- `king - man + woman`  
- `tokyo + france - japan`  
- `apple - technology`  

1. Construya la expresión vectorial que haya definido.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Redacte una conclusión explicando si los resultados corresponden a su hipótesis inicial.